In [1]:
import sys
sys.path.append('/Users/IsabelGontijo/opt/miniconda3/lib/python3.9')

In [2]:
import cidacs
from cidacs import *

Tempo inicial da execucao: 2022-01-13 15:24:32.741743
User: IsabelGontijo
Node: Mac-mini-de-Isabel.local


In [3]:
id_path = '/Users/IsabelGontijo/git/covid-19/refined/data/02_id_creation/'
std_path = '/Users/IsabelGontijo/git/covid-19/refined/data/03_standardization/'
dic_file = '/Users/IsabelGontijo/git/covid-19/refined/metadata/03_standardization/dic.csv'

In [4]:
df_id = spark.read.csv(id_path, header = True, multiLine = True)
df_std = spark.read.parquet(std_path)

In [5]:
#Load dictionary
dic = spark.read.csv(dic_file, header=False, multiLine = True)
bytes = [x._c0.strip().lower() for x in dic.filter((dic['_c1'] == 'Byte')).collect() if x._c0.strip().lower() in df_id.columns]
dates = [x._c0.strip().lower() for x in dic.filter((dic['_c1'] == 'Date')).collect() if x._c0.strip().lower() in df_id.columns]

In [6]:
print(df_id.printSchema())
print(df_std.printSchema())

root
 |-- epi_week: string (nullable = true)
 |-- date: string (nullable = true)
 |-- state: string (nullable = true)
 |-- city: string (nullable = true)
 |-- ibgeID: string (nullable = true)
 |-- newDeaths: string (nullable = true)
 |-- deaths: string (nullable = true)
 |-- newCases: string (nullable = true)
 |-- totalCases: string (nullable = true)
 |-- deaths_per_100k_inhabitants: string (nullable = true)
 |-- totalCases_per_100k_inhabitants: string (nullable = true)
 |-- deaths_by_totalCases: string (nullable = true)
 |-- id: string (nullable = true)

None
root
 |-- epi_week: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- state: byte (nullable = true)
 |-- city: string (nullable = true)
 |-- ibgeid: integer (nullable = true)
 |-- newdeaths: integer (nullable = true)
 |-- deaths: integer (nullable = true)
 |-- newcases: integer (nullable = true)
 |-- totalcases: integer (nullable = true)
 |-- deaths_per_100k_inhabitants: double (nullable = true)
 |-- totalcases_per

In [7]:
print("Existem colunas diferentes: ", set(df_std.columns) - set(df_id.columns) )

Existem colunas diferentes:  {'newcases', 'ibgeid', 'newdeaths', 'deaths_by_totalcases', 'totalcases', 'totalcases_per_100k_inhabitants'}


In [8]:
print("Quantidade de registros antes da padronização: ",df_id.count())
print("Quantidade de registros após padronização: ",df_std.count())

Quantidade de registros antes da padronização:  3424340
Quantidade de registros após padronização:  3424340


In [9]:
for column in bytes:
    print(column)
    df_id_Null = df_id.filter(df_id[column].isNull()).count()
    df_std_Null =  df_std.filter(df_std[column].isNull()).count()
    print("nulos ->  df_id:  {}   -    df_std:  {}".format(df_id_Null,df_std_Null))
    df_std.groupBy(column).agg({'*':'count'}).select(column,F.col('count(1)')).orderBy(F.col(column).desc()).show(200)
    df_id.groupBy(column).agg({'*':'count'}).select(column,F.col('count(1)')).orderBy(F.col(column).desc()).show(200)
    print()

state
nulos ->  df_id:  0   -    df_std:  0
+-----+--------+
|state|count(1)|
+-----+--------+
|   53|     677|
|   52|  148005|
|   51|   85764|
|   50|   48352|
|   43|  298122|
|   42|  182000|
|   41|  242141|
|   35|  400370|
|   33|   59825|
|   32|   50368|
|   31|  510856|
|   29|  256074|
|   28|   47191|
|   27|   63748|
|   26|  117051|
|   25|  136751|
|   24|  103815|
|   23|  117302|
|   22|  135650|
|   21|  134997|
|   17|   85332|
|   16|   10779|
|   15|   91300|
|   14|   10145|
|   13|   39921|
|   12|   14388|
|   11|   32728|
|    1|     688|
+-----+--------+

+-----+--------+
|state|count(1)|
+-----+--------+
|TOTAL|     688|
|   TO|   85332|
|   SP|  400370|
|   SE|   47191|
|   SC|  182000|
|   RS|  298122|
|   RR|   10145|
|   RO|   32728|
|   RN|  103815|
|   RJ|   59825|
|   PR|  242141|
|   PI|  135650|
|   PE|  117051|
|   PB|  136751|
|   PA|   91300|
|   MT|   85764|
|   MS|   48352|
|   MG|  510856|
|   MA|  134997|
|   GO|  148005|
|   ES|   50368|
|  

In [10]:
for column in dates:
    print(column)
    df_id_Null = df_id.filter(df_id[column].isNull()).count()
    df_std_Null =  df_std.filter(df_std[column].isNull()).count()
    print("nulos ->  df_id:  {}   -    df_std:  {}".format(df_id_Null,df_std_Null))
    df_std.groupBy(F.year(column)).agg({'*':'count'}).select('year('+column+')',F.col('count(1)')).orderBy(F.col('year('+column+')').desc()).show(200)
    #Change this line to manipulate the date as a string depending on the format.
    #df.select(F.substring(df[column], 5, 4).alias(column)).groupBy(column).count().orderBy(F.col('count').desc()).show()
    print()

date
nulos ->  df_id:  0   -    df_std:  0
+----------+--------+
|year(date)|count(1)|
+----------+--------+
|      2022|   67164|
|      2021| 2042733|
|      2020| 1314443|
+----------+--------+




In [11]:
cidacs.executionTime()

Tempo de execucao ate este ponto: 0:00:26.929696


In [12]:
sc.stop()